# Preprocessing the mp3

In [1]:
from tqdm.notebook import tqdm, trange
import time    # to be used in loop iterations
from pydub import AudioSegment
import os
import IPython.display as ipd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Convert mp3 to numpy arrays

In [2]:
mp3_paths = list()
for f_year in os.listdir(".mp3"):
    files = os.listdir(f".mp3/{f_year}")
    for f in files:
        mp3_paths.append( f".mp3/{f_year}/{f}")
len(mp3_paths)

7000

In [3]:
mp3_paths[:5]

['.mp3/2013/5RCD9gtzRdLjxcm0EJfic2.mp3',
 '.mp3/2013/6gUcmw16g0gACKICSTIowi.mp3',
 '.mp3/2013/6RMpR8v0WmlIhChhua2hso.mp3',
 '.mp3/2013/3U4isOIWM3VvDubwSI3y7a.mp3',
 '.mp3/2013/2hYarNjMU7pZVY9QsIJIJ3.mp3']

In [4]:
sr = 16000

In [9]:
def load_sound(file, start=0, sr=sr):
    sound = AudioSegment.from_file(file,format="mp3",start_second=start)
    sound = sound.set_frame_rate(sr)
    sound = sound.set_channels(1)
    samples = sound.get_array_of_samples()
    fp_arr = np.array(samples).T.astype(np.float32)
    return fp_arr

In [18]:
# transform mp3 to np.array and save it into the folder .npy/year/file
def audio_generator(mp3_paths, sr=sr):
    for i in trange(len(mp3_paths)):
        path = mp3_paths[i]
        [_,year,_] = path.split("/")
        arr = load_sound(path)
        l = len(arr)
        for segment in range(0, l, 5*sr):
            yield arr[segment:segment+5*sr], year

In [19]:
audio_gen = audio_generator(mp3_paths)

## Extract audio features with autoencoder vgg

In [20]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the model.
model = hub.load('https://tfhub.dev/google/vggish/1')

In [21]:
def vgg_encode(arr):
    embeddings = model(arr)
    embeddings.shape.assert_is_compatible_with([None, 128])
    return embeddings

In [22]:
def generate_features(audios):
    for arr,year in audios:
        yield vgg_encode(arr), year

In [23]:
feat_gen = generate_features(audio_gen)

In [24]:
X = []
y = []
for data_point,label in feat_gen:
    X.append(data_point)
    y.append(label)

  0%|          | 0/7000 [00:00<?, ?it/s]

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [42]:
X_r = [x.reshape(1,-1) for x in X]

In [ ]:
len(X_r), len(y)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_r,y)


In [47]:
len(X_r)

42019

In [45]:
model = KNeighborsClassifier()

In [46]:
model.fit(X_train, y_train)

ValueError: could not broadcast input array from shape (640) into shape (1)

In [27]:
def metrics(model, X=X_test, y=y_test, average="macro"):
    metrics = {}
    functions = {'accuracy': accuracy_score, 
                 'precision': lambda *args: precision_score(*args,average=average, zero_division=1), 
                 'recall': lambda *args: recall_score(*args,average=average), 
                 'f_score': lambda *args: f1_score(*args,average=average)}
    model.fit(X_train, y_train)
    y_pred = model.predict(X)
    for n,m in functions.items():
        metrics[n] = m(y,y_pred)
    return metrics

In [28]:
metrics(KNeighborsClassifier())

ValueError: Expected 2D array, got 1D array instead:
array=[<tf.Tensor: shape=(5, 128), dtype=float32, numpy=
array([[-1.60272515e+00, -3.03061575e-01,  3.10289085e-01,
        -1.15565002e+00, -1.39301860e+00, -2.08842659e+00,
         2.39799857e-01, -5.21546185e-01, -3.74290895e+00,
        -1.02482510e+00,  1.06725335e-01, -9.06804740e-01,
        -1.16349804e+00,  5.70958853e-01, -4.21124518e-01,
         1.21533513e+00,  1.58142462e-01,  4.29520011e-02,
        -6.75586641e-01,  5.69330156e-01, -7.74140477e-01,
        -3.57936025e-02, -1.40851021e+00, -4.29768234e-01,
        -4.17944193e-01,  1.32709491e+00,  1.05387107e-01,
         4.43488732e-03,  5.25538683e-01, -7.24825203e-01,
         8.69718730e-01, -2.67527044e-01,  7.26659775e-01,
        -6.01065084e-02,  7.22436249e-01, -3.18102419e-01,
        -3.77149761e-01, -2.24865198e+00, -1.20720971e+00,
        -5.83818555e-02,  4.35439795e-01, -1.76575541e+00,
         8.74794602e-01, -9.32012498e-01,  4.30910289e-02,
     